In [1]:
from cadcad_gpt import CadCAD_GPT

In [2]:
import pandas as pd
import numpy as np
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend

system_params = {
    'prey_reproduction_rate': [0.6],
    'predator_interaction_factor': [0.0002],
    'prey_interaction_factor': [0.002],
    'prey_death_rate': [0.02],
    'predator_death_rate': [1.0],
    'dt': [0.1]
}

initial_state = {
    'prey_population': 100,
    'predator_population': 50
}


def p_reproduce_prey(params, substep, state_history, prev_state, **kwargs):
    born_population = prev_state['prey_population'] * params['prey_reproduction_rate'] * params['dt']
    return {'add_prey': born_population}


def p_reproduce_predators(params, substep, state_history, prev_state, **kwargs):
    born_population = prev_state['predator_population'] * prev_state['prey_population'] * params['predator_interaction_factor'] * params['dt']
    return {'add_predators': born_population}


def p_eliminate_prey(params, substep, state_history, prev_state, **kwargs):
    population = prev_state['prey_population']
    natural_elimination = population * params['prey_death_rate']
    
    interaction_elimination = population * prev_state['predator_population'] * params['prey_interaction_factor']
    
    eliminated_population = natural_elimination + interaction_elimination * params['dt']
    return {'add_prey': -1.0 * eliminated_population}


def p_eliminate_predators(params, substep, state_history, prev_state, **kwargs):
    population = prev_state['predator_population']
    eliminated_population = population * params['predator_death_rate'] * params['dt']
    return {'add_predators': -1.0 * eliminated_population}


def s_prey_population(params, substep, state_history, prev_state, policy_input, **kwargs):
    updated_prey_population = np.ceil(prev_state['prey_population'] + policy_input['add_prey'])
    return ('prey_population', max(updated_prey_population, 0))


def s_predator_population(params, substep, state_history, prev_state, policy_input, **kwargs):
    updated_predator_population = np.ceil(prev_state['predator_population'] + policy_input['add_predators'])
    return ('predator_population', max(updated_predator_population, 0))


state_update_blocks = [
    {
        'policies': {
            'reproduce_prey': p_reproduce_prey,
            'reproduce_predators': p_reproduce_predators,
            'eliminate_prey': p_eliminate_prey,
            'eliminate_predators': p_eliminate_predators
        },
        'variables': {
            'prey_population': s_prey_population,
            'predator_population': s_predator_population            
        }
    }

]


TIMESTEPS = 1000
RUNS = 1


model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)
experiment = Experiment(simulation)
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

# result = experiment.run()

In [3]:
docs = 'hi i am docs'

In [4]:
# apiu key from .env using dotenv
from dotenv import load_dotenv
import os
load_dotenv()
openai_key = os.getenv('OPENAI_KEY')

In [5]:
cadcad_gpt = CadCAD_GPT(model,simulation, experiment,docs, openai_key)

In [6]:
cadcad_gpt('hi can you change the prey reproduction rate to 0.5?')

Planner Agent:
I have made a plan to follow:
 1) use the function change_param to change the prey reproduction rate parameter to 0.5. 


Executor Agent:
Thought: My task is to  1) use the function change_param to change the prey reproduction rate parameter to 0.5. 
Action: I should call change_param function with these {'param': 'prey_reproduction_rate', 'value': '0.5'} arguments
Observation:  new prey_reproduction_rate value is 0.5 and the simulation dataframe is updated


In [7]:
import plotly.express as px

In [8]:
cadcad_gpt('hi can you change the prey reproduction rate to 0?')

Planner Agent:
I have made a plan to follow:
 1) Use the function change_param to change the prey reproduction rate to 0.
2) Run the model to see the effects of the change. 


Executor Agent:
Thought: My task is to  1) Use the function change_param to change the prey reproduction rate to 0.
Action: I should call change_param function with these {'param': 'prey_reproduction_rate', 'value': '0'} arguments
Observation:  new prey_reproduction_rate value is 0.0 and the simulation dataframe is updated
Executor Agent:
Thought: My task is to 2) Run the model to see the effects of the change. 
Action: I should call change_param function with these {'param': 'prey_reproduction_rate', 'value': '0.2'} arguments
Observation:  new prey_reproduction_rate value is 0.2 and the simulation dataframe is updated


In [10]:
df = pd.DataFrame(experiment.run())
df.head()

,prey_population,predator_population,simulation,subset,run,substep,timestep
0,100.0,50.0,0,0,1,0,0
1,99.0,46.0,0,0,1,1,1
2,99.0,42.0,0,0,1,1,2
3,99.0,38.0,0,0,1,1,3
4,99.0,35.0,0,0,1,1,4


In [1]:
from utils import schema

In [11]:
from pydantic import create_model
import inspect
from inspect import Parameter
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
        for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)


# def sums(a:int, b:int=1):
#     "Adds a + b"
#     return a + b

# schema(sums)

# convert the function schema to a python decorator called @tool that can be used to convert the functions to a class which has a method called schema
def tool(f):
    def wrapper(*args, **kwargs):
        return f(*args, **kwargs)
    wrapper.schema = schema(f)
    return wrapper

@tool
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b


In [13]:
@tool
def analyze_dataframe(question:str)->str:
    '''Use this whenever a quantitative question is asked about the dataframe. The question should be taken exactly as asked by the user'''
    # Implement your logic here to analyze the dataframe based on the user's question
    return question

In [14]:
analyze_dataframe.schema

{'name': 'analyze_dataframe',
 'description': 'Use this whenever a quantitative question is asked about the dataframe. The question should be taken exactly as asked by the user',
 'parameters': {'title': 'Input for `analyze_dataframe`',
  'type': 'object',
  'properties': {'question': {'title': 'Question', 'type': 'string'}},
  'required': ['question']}}

In [17]:
@tool
def change_param(param:str, value:float)->str:
    '''Changes the parameter of the cadcad simulation and returns dataframe as a global object. The parameter must be in this list:'''
    if param not in self.model.params:
        return f'{param} is not a parameter of the model'
    value = float(value)
    self.simulation.model.params.update({
        param: [value]
    })
    self.experiment = Experiment(self.simulation)
    self.experiment.engine = Engine()
    result = self.experiment.run()
    # Convert the results to a pandas DataFrame
    self.df = pd.DataFrame(result)
    return f'new {param} value is {value} and the simulation dataframe is updated'

In [ ]:
{
    "name": "change_param",
    "description": "Changes the parameter of the cadcad simulation and returns dataframe as a global object. The parameter must be in this list:" + str(model.params.keys()),
    "parameters": {
        "type": "object",
        "properties": {
            "param": {
                "type": "string",
                "description": "parameter to change. choose from the list" ,
            },
            "value": {
                "type": "string",
                "description": "value to change the parameter to, eg. 0.1",
            },
        },
        "required": ["param", "value"],
    }
}